**K=10**

initialization of k-fold function

In [24]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import RFE
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, roc_auc_score, f1_score
merged_df3=pd.read_csv('btc_only_df.csv')

#time series k fold cross validation split where k=10
import math

def k_fold_time_series_blocking(k=10):
    merged_df3.reset_index(drop=True, inplace=True)
    n=len(merged_df3)
    fold_size=n//k
    train_size= math.floor(0.8 * fold_size)
    test_size=math.floor(0.2*fold_size)

    indexes=[]

    for i in range(k):
        start_train=i*fold_size
        end_train=start_train+train_size-1
        start_test=end_train+1
        end_test=start_test+test_size-1
        yield np.arange(start_train, end_train, dtype=int),np.arange(start_test, end_test, dtype=int)

In [25]:
max_features=len(merged_df3.drop(columns=['Date','Close2','Close2-Close1','next_day_close_increased']).columns)


In [26]:
print(max_features)

16


**General structure for each type of models**

We will be automating the hyperparameter tuning (using GridSearch) and feature selection process (using SelectKbest)


for each of the type of models below:

1.Select n best features based on mutual_info_regression

2.pass these n features into the model and conduct grid search with cross-validation , select the best hyperparameters based on aggregated f1 score over k folds

3.for the best esimator , print its hyperparameters ,features selected as well as scores

4.repeat for different n ( 3 to max_number of feature columns in dataset)

**SVM-poly kernel**




In [27]:
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.pipeline import Pipeline
def svmPolypipeline(n):


  pipeline = Pipeline([
      ('feature_selection', SelectKBest(mutual_info_regression,k=n)),
      ('svm', SVC(kernel='poly'))
  ])

  #  Define parameter grid for GridSearchCV
  param_grid = {

      'svm__C': [0.1, 1, 10],                # Regularization parameter
      'svm__degree': [2, 3, 4],              # Degree of the polynomial kernel
      'svm__coef0': [0.0, 1.0, 2.0]          # Independent term in the polynomial kernel
  }
  f1_scorer = make_scorer(f1_score)
  roc_auc_scorer = make_scorer(roc_auc_score)

  input=merged_df3.drop(columns=['Date','Close2','Close2-Close1','next_day_close_increased'])
  out=merged_df3['next_day_close_increased']

  # Perform grid search with cross-validation
  #select best estimator using aggregated f1 score over all k fold as defined by refit='f1'
  grid_search = GridSearchCV(pipeline, param_grid=param_grid, cv=k_fold_time_series_blocking(),scoring={'f1': f1_scorer, 'roc_auc': roc_auc_scorer}, refit='f1', verbose=0)
  grid_result = grid_search.fit(input, out)

  #retrieving and printing the selected features and hyperparameters
  selected_feature_names=[]
  selected_features_bool=grid_result.best_estimator_.named_steps['feature_selection'].get_support()
  for i in range(len(selected_features_bool)):
    if(selected_features_bool[i]):
      selected_feature_names.append(input.columns[i])

  print("Selected Features:", selected_feature_names)

  print("Best F1 Score:", grid_result.best_score_) #print best score
  print("Best Parameters:", grid_result.best_params_) #print best hyperparameters

  #get best weights
  best_estimator = grid_result.best_estimator_

  # Access the RandomForestClassifier from the best estimator
  svm_model = best_estimator.named_steps['svm']


  return svm_model,grid_result.best_score_,selected_feature_names


In [28]:
all_weights={}
all_selected_features={}
max_f1_index=0
max_f1=0

for i in range(3,16):
  print("i: ",i)
  weights,best_f1,selected_features=svmPolypipeline(i) #i is number of features

  max_f1=max(max_f1,best_f1)
  if max_f1==best_f1:
    max_f1_index=i

  all_weights[i]=weights
  all_selected_features[i]=selected_features


print("index:",max_f1_index)
print("max_f1:",max_f1)

i:  3
Selected Features: ['Volume', 'macd', 'OBV']
Best F1 Score: 0.3674303405572756
Best Parameters: {'svm__C': 10, 'svm__coef0': 0.0, 'svm__degree': 3}
i:  4
Selected Features: ['Volume', 'macd', 'macd_signal', 'OBV']
Best F1 Score: 0.4311589580010632
Best Parameters: {'svm__C': 10, 'svm__coef0': 2.0, 'svm__degree': 4}
i:  5
Selected Features: ['Volume', 'macd', 'macd_signal', 'High-Low', 'OBV']
Best F1 Score: 0.33612010796221325
Best Parameters: {'svm__C': 10, 'svm__coef0': 1.0, 'svm__degree': 4}
i:  6
Selected Features: ['High', 'Volume', 'macd', 'macd_signal', 'High-Low', 'OBV']
Best F1 Score: 0.34237516869095813
Best Parameters: {'svm__C': 1, 'svm__coef0': 0.0, 'svm__degree': 4}
i:  7
Selected Features: ['High', 'Adj Close', 'Volume', 'macd', 'macd_signal', 'High-Low', 'OBV']
Best F1 Score: 0.3698965362123257
Best Parameters: {'svm__C': 1, 'svm__coef0': 2.0, 'svm__degree': 2}
i:  8
Selected Features: ['High', 'Close', 'Adj Close', 'Volume', 'macd', 'macd_signal', 'High-Low', 'OBV

In [29]:
import pickle
poly_model=all_weights[max_f1_index]
with open('poly_svm_model_k10_BTC_only.pkl', 'wb') as f:
    pickle.dump(poly_model, f)

In [31]:
# Use the loaded model to make predictions on new data
import pickle
from sklearn.metrics import classification_report

df_backtest=pd.read_csv('BTC-USD-backtest-cleaned-final.csv')#backtest dataset

with open('poly_svm_model_k10_BTC_only.pkl', 'rb') as f:
    trained_model = pickle.load(f)


selected_feats=all_selected_features[max_f1_index]
print(selected_feats)
X_test = df_backtest[selected_feats]
y_true=df_backtest['next_day_close_increased']
predictions = trained_model.predict(X_test.values)

# Generate the classification report
report = classification_report(y_true, predictions)

# Print the classification report
print(report)

['Open', 'High', 'Close', 'Adj Close', 'Volume', 'RSI_14', 'macd', 'macd_signal', 'BB_lower', 'High-Low', 'OBV', 'BB_height']
              precision    recall  f1-score   support

           0       0.67      0.67      0.67        18
           1       0.54      0.54      0.54        13

    accuracy                           0.61        31
   macro avg       0.60      0.60      0.60        31
weighted avg       0.61      0.61      0.61        31



**SVM-RBF**

In [32]:
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.pipeline import Pipeline
def svmRBFpipeline(n):
  pipeline = Pipeline([
      ('feature_selection', SelectKBest(mutual_info_regression,k=n)),
      ('svm', SVC(kernel='rbf'))
  ])
  param_grid = {

      'svm__C': [0.1, 1, 10],                # Regularization parameter
      'svm__gamma': [0.1, 1, 10, 100],

  }

  f1_scorer = make_scorer(f1_score)
  roc_auc_scorer = make_scorer(roc_auc_score)

  input=merged_df3.drop(columns=['Date','Close2','Close2-Close1','next_day_close_increased'])
  out=merged_df3['next_day_close_increased']

  # 5. Perform grid search with cross-validation
  grid_search = GridSearchCV(pipeline, param_grid=param_grid, cv=k_fold_time_series_blocking(),scoring={'f1': f1_scorer, 'roc_auc': roc_auc_scorer}, refit='f1', verbose=0)
  grid_result = grid_search.fit(input, out)

  selected_feature_names=[]
  selected_features_bool=grid_result.best_estimator_.named_steps['feature_selection'].get_support()
  for i in range(len(selected_features_bool)):
    if(selected_features_bool[i]):
      selected_feature_names.append(input.columns[i])

  print("Selected Features:", selected_feature_names)

  print("Best F1 Score:", grid_result.best_score_)
  print("Best Parameters:", grid_result.best_params_)
  #get best weights
  best_estimator = grid_result.best_estimator_

  # Access the RandomForestClassifier from the best estimator
  svm_model = best_estimator.named_steps['svm']


  return svm_model,grid_result.best_score_,selected_feature_names

In [33]:
all_weights={}
all_selected_features={}
max_f1_index=0
max_f1=0

for i in range(3,16):
  print("i: ",i)
  weights,best_f1,selected_features=svmRBFpipeline(i) #i is number of features

  max_f1=max(max_f1,best_f1)
  if max_f1==best_f1:
    max_f1_index=i

  all_weights[i]=weights
  all_selected_features[i]=selected_features


print("index:",max_f1_index)
print("max_f1:",max_f1)

i:  3
Selected Features: ['Volume', 'macd', 'OBV']
Best F1 Score: 0.3230974708724069
Best Parameters: {'svm__C': 10, 'svm__gamma': 1}
i:  4
Selected Features: ['Volume', 'macd', 'macd_signal', 'OBV']
Best F1 Score: 0.4549443461595164
Best Parameters: {'svm__C': 10, 'svm__gamma': 100}
i:  5
Selected Features: ['Volume', 'macd', 'macd_signal', 'High-Low', 'OBV']
Best F1 Score: 0.3533752496213796
Best Parameters: {'svm__C': 10, 'svm__gamma': 100}
i:  6
Selected Features: ['High', 'Volume', 'macd', 'macd_signal', 'High-Low', 'OBV']
Best F1 Score: 0.3905955877627704
Best Parameters: {'svm__C': 10, 'svm__gamma': 100}
i:  7
Selected Features: ['High', 'Adj Close', 'Volume', 'macd', 'macd_signal', 'High-Low', 'OBV']
Best F1 Score: 0.4421129360835243
Best Parameters: {'svm__C': 10, 'svm__gamma': 100}
i:  8
Selected Features: ['High', 'Close', 'Adj Close', 'Volume', 'macd', 'macd_signal', 'High-Low', 'OBV']
Best F1 Score: 0.3992557932263815
Best Parameters: {'svm__C': 10, 'svm__gamma': 100}
i:  

In [34]:
import pickle
rbf_model=all_weights[max_f1_index]
with open('rbf_svm_model_k10_BTC_only.pkl', 'wb') as f:
    pickle.dump(rbf_model, f)

In [35]:
# Use the loaded model to make predictions on new data

from sklearn.metrics import classification_report

df_backtest=pd.read_csv('BTC-USD-backtest-cleaned-final.csv')#backtest dataset

with open('rbf_svm_model_k10_BTC_only.pkl', 'rb') as f:
    trained_model = pickle.load(f)


selected_feats=all_selected_features[max_f1_index]
X_test = df_backtest[selected_feats]
y_true=df_backtest['next_day_close_increased']
predictions = trained_model.predict(X_test.values)

# Generate the classification report
report = classification_report(y_true, predictions)

# Print the classification report
print(report)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        18
           1       0.42      1.00      0.59        13

    accuracy                           0.42        31
   macro avg       0.21      0.50      0.30        31
weighted avg       0.18      0.42      0.25        31



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**RF**

In [36]:
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
def RFpipeline(n):

  pipeline = Pipeline([
      ('feature_selection', SelectKBest(mutual_info_regression,k=n)),
      ('RF', RandomForestClassifier())
  ])

  #  Define parameter grid for GridSearchCV
  param_grid = {
      'RF__n_estimators': [50, 100, 200],
      'RF__max_depth': [None, 10, 20],
      'RF__min_samples_split': [2, 5, 10],
      'RF__min_samples_leaf': [1, 2, 4]
  }
  f1_scorer = make_scorer(f1_score)
  roc_auc_scorer = make_scorer(roc_auc_score)

  input=merged_df3.drop(columns=['Date','Close2','Close2-Close1','next_day_close_increased'])
  out=merged_df3['next_day_close_increased']

  # 5. Perform grid search with cross-validation
  grid_search = GridSearchCV(pipeline, param_grid=param_grid, cv=k_fold_time_series_blocking(),scoring={'f1': f1_scorer, 'roc_auc': roc_auc_scorer}, refit='f1', verbose=0)
  grid_result = grid_search.fit(input, out)

  selected_feature_names=[]
  selected_features_bool=grid_result.best_estimator_.named_steps['feature_selection'].get_support()
  for i in range(len(selected_features_bool)):
    if(selected_features_bool[i]):
      selected_feature_names.append(input.columns[i])

  print("Selected Features:", selected_feature_names)

  print("Best F1 Score:", grid_result.best_score_)
  print("Best Parameters:", grid_result.best_params_)
  #get best weights
  best_estimator = grid_result.best_estimator_

  # Access the RandomForestClassifier from the best estimator
  rf_model = best_estimator.named_steps['RF']


  return rf_model,grid_result.best_score_,selected_feature_names

In [37]:
all_weights={}
all_selected_features={}
max_f1_index=0
max_f1=0

for i in range(3,16):
  print("i: ",i)
  weights,best_f1,selected_features=RFpipeline(i) #i is number of features

  max_f1=max(max_f1,best_f1)
  if max_f1==best_f1:
    max_f1_index=i

  all_weights[i]=weights
  all_selected_features[i]=selected_features


print("index:",max_f1_index)
print("max_f1:",max_f1)

i:  3
Selected Features: ['Volume', 'macd', 'OBV']
Best F1 Score: 0.321038961038961
Best Parameters: {'RF__max_depth': None, 'RF__min_samples_leaf': 1, 'RF__min_samples_split': 2, 'RF__n_estimators': 50}
i:  4
Selected Features: ['Volume', 'macd', 'macd_signal', 'OBV']
Best F1 Score: 0.38831932773109246
Best Parameters: {'RF__max_depth': 20, 'RF__min_samples_leaf': 2, 'RF__min_samples_split': 10, 'RF__n_estimators': 50}
i:  5
Selected Features: ['Volume', 'macd', 'macd_signal', 'High-Low', 'OBV']
Best F1 Score: 0.420982905982906
Best Parameters: {'RF__max_depth': None, 'RF__min_samples_leaf': 4, 'RF__min_samples_split': 5, 'RF__n_estimators': 200}
i:  6
Selected Features: ['High', 'Volume', 'macd', 'macd_signal', 'High-Low', 'OBV']
Best F1 Score: 0.41663861766028953
Best Parameters: {'RF__max_depth': 10, 'RF__min_samples_leaf': 4, 'RF__min_samples_split': 5, 'RF__n_estimators': 50}
i:  7
Selected Features: ['High', 'Adj Close', 'Volume', 'macd', 'macd_signal', 'High-Low', 'OBV']
Best F

In [38]:
import pickle
rf_model=all_weights[max_f1_index]
with open('random_forest_model_k10_BTC_only.pkl', 'wb') as f:
    pickle.dump(rf_model, f)

In [39]:
# Use the loaded model to make predictions on new data

from sklearn.metrics import classification_report

df_backtest=pd.read_csv('BTC-USD-backtest-cleaned-final.csv')#backtest dataset

with open('random_forest_model_k10_BTC_only.pkl', 'rb') as f:
    trained_model = pickle.load(f)


selected_feats=all_selected_features[max_f1_index]
X_test = df_backtest[selected_feats]
y_true=df_backtest['next_day_close_increased']
predictions = trained_model.predict(X_test.values)

# Generate the classification report
report = classification_report(y_true, predictions)

# Print the classification report
print(report)

              precision    recall  f1-score   support

           0       0.67      0.56      0.61        18
           1       0.50      0.62      0.55        13

    accuracy                           0.58        31
   macro avg       0.58      0.59      0.58        31
weighted avg       0.60      0.58      0.58        31



**Adaboost**

In [40]:
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.pipeline import Pipeline
from sklearn.ensemble import AdaBoostClassifier
def adaboostpipeline(n):
  pipeline =  Pipeline([
      ('feature_selection', SelectKBest(mutual_info_regression,k=n)),
      ('adaboost', AdaBoostClassifier()) ])

  # 4. Define parameter grid for GridSearchCV
  param_grid = {

      'adaboost__n_estimators': [50, 100, 150], # Number of weak learners
      'adaboost__learning_rate': [0.1, 0.5, 1.0] # Learning rate
  }
  f1_scorer = make_scorer(f1_score)
  roc_auc_scorer = make_scorer(roc_auc_score)

  input=merged_df3.drop(columns=['Date','Close2','Close2-Close1','next_day_close_increased'])
  out=merged_df3['next_day_close_increased']

  # 5. Perform grid search with cross-validation
  grid_search = GridSearchCV(pipeline, param_grid=param_grid, cv=k_fold_time_series_blocking(),scoring={'f1': f1_scorer, 'roc_auc': roc_auc_scorer}, refit='f1', verbose=0)
  grid_result = grid_search.fit(input, out)

  selected_feature_names=[]
  selected_features_bool=grid_result.best_estimator_.named_steps['feature_selection'].get_support()
  for i in range(len(selected_features_bool)):
    if(selected_features_bool[i]):
      selected_feature_names.append(input.columns[i])

  print("Selected Features:", selected_feature_names)

  print("Best F1 Score:", grid_result.best_score_)
  print("Best Parameters:", grid_result.best_params_)
    #get best weights
  best_estimator = grid_result.best_estimator_

  # Access the RandomForestClassifier from the best estimator
  ada_model = best_estimator.named_steps['adaboost']


  return ada_model,grid_result.best_score_,selected_feature_names

In [41]:
all_weights={}
all_selected_features={}
max_f1_index=0
max_f1=0

for i in range(3,16):
  print("i: ",i)
  weights,best_f1,selected_features=adaboostpipeline(i) #i is number of features

  max_f1=max(max_f1,best_f1)
  if max_f1==best_f1:
    max_f1_index=i

  all_weights[i]=weights
  all_selected_features[i]=selected_features


print("index:",max_f1_index)
print("max_f1:",max_f1)

i:  3
Selected Features: ['Volume', 'macd', 'OBV']
Best F1 Score: 0.38178042030209214
Best Parameters: {'adaboost__learning_rate': 1.0, 'adaboost__n_estimators': 100}
i:  4
Selected Features: ['Volume', 'macd', 'macd_signal', 'OBV']
Best F1 Score: 0.4092862039920863
Best Parameters: {'adaboost__learning_rate': 1.0, 'adaboost__n_estimators': 150}
i:  5
Selected Features: ['Volume', 'macd', 'macd_signal', 'High-Low', 'OBV']
Best F1 Score: 0.3352852049910874
Best Parameters: {'adaboost__learning_rate': 1.0, 'adaboost__n_estimators': 50}
i:  6
Selected Features: ['High', 'Volume', 'macd', 'macd_signal', 'High-Low', 'OBV']
Best F1 Score: 0.3722549019607843
Best Parameters: {'adaboost__learning_rate': 1.0, 'adaboost__n_estimators': 50}
i:  7
Selected Features: ['High', 'Adj Close', 'Volume', 'macd', 'macd_signal', 'High-Low', 'OBV']
Best F1 Score: 0.36236768391567153
Best Parameters: {'adaboost__learning_rate': 1.0, 'adaboost__n_estimators': 50}
i:  8
Selected Features: ['High', 'Close', 'Ad

In [42]:
import pickle
ada_model=all_weights[max_f1_index]
with open('ada_model_k10_BTC_only.pkl', 'wb') as f:
    pickle.dump(ada_model, f)

In [43]:
# Use the loaded model to make predictions on new data

from sklearn.metrics import classification_report

df_backtest=pd.read_csv('BTC-USD-backtest-cleaned-final.csv')#backtest dataset

with open('ada_model_k10_BTC_only.pkl', 'rb') as f:
    trained_model = pickle.load(f)


selected_feats=all_selected_features[max_f1_index]
X_test = df_backtest[selected_feats]
y_true=df_backtest['next_day_close_increased']
predictions = trained_model.predict(X_test.values)

# Generate the classification report
report = classification_report(y_true, predictions)

# Print the classification report
print(report)

              precision    recall  f1-score   support

           0       0.58      0.39      0.47        18
           1       0.42      0.62      0.50        13

    accuracy                           0.48        31
   macro avg       0.50      0.50      0.48        31
weighted avg       0.52      0.48      0.48        31

